In [1]:
# Install dependencies

import sys
!{sys.executable} -m pip install hdfs

/opt/conda/bin/python: No module named pip install hdfs


In [2]:
pip install delta-spark

Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/Projeto/Gold'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [4]:
spark.sql(
    """
    DROP TABLE IF EXISTS ProjetoGold.Gold_Inspecoes
    """
)

spark.sql(
    
    
    """
    CREATE EXTERNAL TABLE ProjetoGold.Gold_Inspecoes (
    
        Site_Name string,
        Boro string,
        District string,
        Overall_Rating string,
        Cleanliness_Rating string,
        Year int,
        Data string,
        PIP_Category string
        )
    
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/Projeto/Gold/ProjetoGold.db/Gold_Inspecoes'
    """
    
)

DataFrame[]

In [5]:
from pyspark.sql.functions import substring, avg, sum

# read  Parks_Supervisor_Inspections_Inspection_Results from the silver tables
hdfs_path = 'hdfs://hdfs-nn:9000/Projeto/Silver/Projeto.db/Inspecoes'

Inspecoes = spark\
             .read\
             .load(hdfs_path)



In [ ]:
# write to delta table
Inspecoes \
    .select("Site_Name", "Boro", "District", "Overall_Rating", "Cleanliness_Rating", "Year", "Data", "PIP_Category") \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/Projeto/Gold/ProjetoGold.db/Gold_Inspecoes/")

In [ ]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/Projeto/Gold/ProjetoGold.db/Gold_Inspecoes/`
""").show()

In [ ]:
spark.sql("""
DROP TABLE IF EXISTS ProjetoGold.Gold_Inspecoes_Presto
""").show()

spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS ProjetoGold.Gold_Inspecoes_Presto (
  
    Site_Name string,
    Boro string,
    District string,
    Overall_Rating string,
    Cleanliness_Rating string,
    Year int,
    Data string,
    PIP_Category string
   
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/Projeto/Gold/ProjetoGold.db/Gold_Inspecoes/_symlink_format_manifest/'
""").show()